In [42]:
pip install -U elasticsearch

  Using cached elasticsearch-8.2.3-py3-none-any.whl (381 kB)
ERROR: pyelasticsearch 1.4.1 has requirement elasticsearch<2.0.0,>=1.3.0, but you'll have elasticsearch 8.2.3 which is incompatible.
  Attempting uninstall: elasticsearch
    Found existing installation: elasticsearch 1.9.0
    Uninstalling elasticsearch-1.9.0:
      Successfully uninstalled elasticsearch-1.9.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    import pandas as pd
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    import os
    import sys
    import os
    import numpy as np
    from elasticsearch import helpers
    print("Loaded ... ... ...")
except Exception as e:
    print("Some Modules are Missing{}".format(e))

Loaded ... ... ...


In [2]:
for x in os.listdir():
    print(x)

.ipynb_checkpoints
elasticsearch.py.ipynb
elasticsearch.ipynb
Elastic_Search_Engine.ipynb
netflix_titles.csv
data_job.csv
.git


### read the dataset

In [3]:
df = pd.read_csv('netflix_titles.csv')
df.sample(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
2208,s2209,Movie,Dragons: Rescue Riders: Secrets of the Songwing,"Greg Rankin, TJ Sullivan","Noah Kaye Bentley, Brennley Brown, Zach Callis...",United States,"July 24, 2020",2020,TV-Y,46 min,"Children & Family Movies, Comedies",When a musical dragon with a beautiful voice h...
8018,s8019,Movie,Silent Hill: Revelation,M.J. Bassett,"Deborah Kara Unger, Adelaide Clemens, Sean Bea...","Canada, France, United States","December 13, 2019",2012,R,94 min,Horror Movies,Haunted by nightmares and her father's disappe...
1719,s1720,Movie,We Are All For The Fatherland,NaN,NaN,"Lebanon, Canada, France","November 11, 2020",1979,TV-14,74 min,"Documentaries, International Movies","After the 1978 Israeli invasion of Lebanon, ch..."
4438,s4439,Movie,Ghanchakkar,Raj Kumar Gupta,"Emraan Hashmi, Vidya Balan, Rajesh Sharma, Nam...",India,"November 1, 2018",2013,TV-MA,129 min,"Comedies, International Movies, Thrillers",Former con artist Sanju surrounds himself with...
4016,s4017,TV Show,Turn Up Charlie,NaN,"Idris Elba, Piper Perabo, JJ Feild, Frankie He...",United Kingdom,"March 15, 2019",2019,TV-MA,1 Season,"British TV Shows, International TV Shows, TV C...",A down-and-out DJ plots to rebuild his music c...


In [4]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

In [5]:
df.shape

(8807, 12)

In [6]:
df["show_id"].nunique()

8807

### Creating a elastic Search Instance

In [7]:
ENDPOINT = "http://localhost:9200/"
es = Elasticsearch(timeout=600,hosts=ENDPOINT)

/tmp/ipykernel_7074/4263798987.py:2: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es = Elasticsearch(timeout=600,hosts=ENDPOINT)


In [8]:
es.ping()

/tmp/ipykernel_7074/2803272245.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.ping()


True

### Before uploading do some data cleaning

In [10]:
df.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [11]:
df = df.dropna()

In [12]:
df.isna().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [13]:
df.shape

(5332, 12)

#### We need to convert the data an appropriate format that elastic search can understand

In [14]:
df2 = df.to_dict('records')

In [15]:
df2[0]

{'show_id': 's8',
 'type': 'Movie',
 'title': 'Sankofa',
 'director': 'Haile Gerima',
 'cast': 'Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra Duah, Nick Medley, Mutabaruka, Afemo Omilami, Reggie Carter, Mzuri',
 'country': 'United States, Ghana, Burkina Faso, United Kingdom, Germany, Ethiopia',
 'date_added': 'September 24, 2021',
 'release_year': 1993,
 'rating': 'TV-MA',
 'duration': '125 min',
 'listed_in': 'Dramas, Independent Movies, International Movies',
 'description': 'On a photo shoot in Ghana, an American model slips back in time, becomes enslaved on a plantation and bears witness to the agony of her ancestral past.'}

In [16]:
len(df2)

5332

In [17]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

## we need to convert the data into ELK format

In [18]:
def generator(df2):
    for c,line in enumerate(df2):
        yield {
    '_index': 'myelkfirst',
    '_type': '_doc',
    '_id': line.get("show_id",None),
    '_source': {       
       'title': line.get("title",""),
       'director':line.get('director',""),
       'description':line.get('description',""),
        'duration':line.get('duration',None),
        'cast': line.get('cast',None)
        }
         }
    raise StopIteration

In [19]:
mycustom = generator(df2)

In [20]:
mycustom

<generator object generator at 0x7f5731fa4890>

In [21]:
next(mycustom)

{'_index': 'myelkfirst',
 '_type': '_doc',
 '_id': 's8',
 '_source': {'title': 'Sankofa',
  'director': 'Haile Gerima',
  'description': 'On a photo shoot in Ghana, an American model slips back in time, becomes enslaved on a plantation and bears witness to the agony of her ancestral past.',
  'duration': '125 min',
  'cast': 'Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra Duah, Nick Medley, Mutabaruka, Afemo Omilami, Reggie Carter, Mzuri'}}

In [22]:
import json
json.dumps(next(mycustom), indent=3)


'{\n   "_index": "myelkfirst",\n   "_type": "_doc",\n   "_id": "s9",\n   "_source": {\n      "title": "The Great British Baking Show",\n      "director": "Andy Devonshire",\n      "description": "A talented batch of amateur bakers face off in a 10-week competition, whipping up their best dishes in the hopes of being named the U.K.\'s best.",\n      "duration": "9 Seasons",\n      "cast": "Mel Giedroyc, Sue Perkins, Mary Berry, Paul Hollywood"\n   }\n}'

## Settings or Mappings

In [23]:
Settings={
   "settings":{
      "number_of_shards":1,
      "number_of_replicas":0
   },
   "mappings":{
      "properties":{
         "director":{
            "type":"text"
         },
         "duration":{
            "type":"text"
         }
      }
   }
}

In [25]:
IndexName = ''
my = es.indices.create(index='myelkfirst', ignore=[400,404], body=Settings)

/tmp/ipykernel_7074/3487492872.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  my = es.indices.create(index='myelkfirst', ignore=[400,404], body=Settings)
/tmp/ipykernel_7074/3487492872.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  my = es.indices.create(index='myelkfirst', ignore=[400,404], body=Settings)
/tmp/ipykernel_7074/3487492872.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  my = es.indices.create(index='myelkfirst', ignore=[400,404], body=Settings)


In [32]:
my

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'myelkfirst'})

## Upload the document into the Elastic Search

In [26]:
try:
    res = helpers.bulk(es, generator(df2))
    print('Working')
except Exception as e:
    pass

/tmp/ipykernel_7074/478267927.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = helpers.bulk(es, generator(df2))
/tmp/ipykernel_7074/478267927.py:2: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  res = helpers.bulk(es, generator(df2))


## Search Query Match Pharse

In [27]:
INPUT = input("Enter the Input Query  : ")

query = {
   "_source": ["title","cast","director","duration"],
   "size":50,
   "query":{
      "match_phrase": {
        "title": INPUT
      }
   }
}
#es = Elasticsearch(timeout=600, hosts=os.getenv("ENDPOINT"))
res = es.search(index='myelkfirst',
                body=query)

title = [x['_source']  for x in res['hits']['hits']]
title

Enter the Input Query Norm of the North


/tmp/ipykernel_7074/35652014.py:13: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index='myelkfirst',
/tmp/ipykernel_7074/35652014.py:13: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index='myelkfirst',


[{'duration': '89 min',
  'cast': 'Andrew Toth, Lisa Durupt, Jonathan Holmes, Paul Dobson, Cole Howard, Brian Drummond, Jennifer Cameron',
  'director': 'Anthony Bell',
  'title': 'Norm of the North: Family Vacation'},
 {'duration': '90 min',
  'cast': 'Alan Marriott, Andrew Toth, Brian Dobson, Cole Howard, Jennifer Cameron, Jonathan Holmes, Lee Tockar, Lisa Durupt, Maya Kay, Michael Dobson',
  'director': 'Richard Finn, Tim Maltby',
  'title': 'Norm of the North: King Sized Adventure'},
 {'duration': '92 min',
  'cast': 'Andrew Toth, Cole Howard, Maya Kay, Jennifer Cameron, Alan Marriott, Lee Tockar, Brian Drummond',
  'director': 'Tim Maltby',
  'title': 'Norm of the North: Keys to the Kingdom'}]

## Search Query for match

In [28]:
INPUT = input("Enter the Input Query  : ")

query = {
   "_source": ["title","cast","director","duration"],
   "size":50,
   "query":{
      "match": {
        "title": INPUT
      }
   }
}
#es = Elasticsearch(timeout=600, hosts=os.getenv("ENDPOINT"))
res = es.search(index='myelkfirst',
                body=query)

title = [x['_source']  for x in res['hits']['hits']]
title

Enter the Input Query  : Norm of the North


/tmp/ipykernel_7074/859594177.py:13: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index='myelkfirst',
/tmp/ipykernel_7074/859594177.py:13: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index='myelkfirst',


[{'duration': '89 min',
  'cast': 'Andrew Toth, Lisa Durupt, Jonathan Holmes, Paul Dobson, Cole Howard, Brian Drummond, Jennifer Cameron',
  'director': 'Anthony Bell',
  'title': 'Norm of the North: Family Vacation'},
 {'duration': '90 min',
  'cast': 'Alan Marriott, Andrew Toth, Brian Dobson, Cole Howard, Jennifer Cameron, Jonathan Holmes, Lee Tockar, Lisa Durupt, Maya Kay, Michael Dobson',
  'director': 'Richard Finn, Tim Maltby',
  'title': 'Norm of the North: King Sized Adventure'},
 {'duration': '92 min',
  'cast': 'Andrew Toth, Cole Howard, Maya Kay, Jennifer Cameron, Alan Marriott, Lee Tockar, Brian Drummond',
  'director': 'Tim Maltby',
  'title': 'Norm of the North: Keys to the Kingdom'},
 {'duration': '99 min',
  'cast': 'Banky Wellington, Rahama Sadau, Kanayo O. Kanayo, Ibrahim Suleiman, Michelle Dede, Adesua Etomi, Hilda Dokubo, Akin Lewis',
  'director': 'Tope Oshin',
  'title': 'Up North'},
 {'duration': '61 min',
  'cast': 'Norm MacDonald',
  'director': 'Liz Plonka',
 

## Search Query for aggregation of word

In [30]:
query = {
  "size":50,
  "aggs": {
    "cast_names": {
      "terms": {
        "field": "cast.keyword",
        "size": 5,
        "order": {
          "_count": "desc"
        }
      }
    }
  }
}
#es = Elasticsearch(timeout=600, hosts=os.getenv("ENDPOINT"))
res = es.search(index='myelkfirst',
                body=query)

title = [x['_source']  for x in res['hits']['hits']]
title

/tmp/ipykernel_7074/2459221246.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index='myelkfirst',
/tmp/ipykernel_7074/2459221246.py:16: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index='myelkfirst',


[{'title': 'Sankofa',
  'director': 'Haile Gerima',
  'description': 'On a photo shoot in Ghana, an American model slips back in time, becomes enslaved on a plantation and bears witness to the agony of her ancestral past.',
  'duration': '125 min',
  'cast': 'Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra Duah, Nick Medley, Mutabaruka, Afemo Omilami, Reggie Carter, Mzuri'},
 {'title': 'The Great British Baking Show',
  'director': 'Andy Devonshire',
  'description': "A talented batch of amateur bakers face off in a 10-week competition, whipping up their best dishes in the hopes of being named the U.K.'s best.",
  'duration': '9 Seasons',
  'cast': 'Mel Giedroyc, Sue Perkins, Mary Berry, Paul Hollywood'},
 {'title': 'The Starling',
  'director': 'Theodore Melfi',
  'description': "A woman adjusting to life after a loss contends with a feisty bird that's taken over her garden — and a husband who's struggling to find a way forward.",
  'duration': '104 min',
  'cast': "Melissa McCarthy, Chr